In [37]:
!grep -o '>' fastaresults.txt | wc -l

   20407


In [9]:
!head -n 50 fastaresults.txt

>ENA|AY220845|AY220845.1 Homo sapiens cytochrome P450 mRNA, complete cds. : Location:1..6000
CACGAGGGGAAGGGTCACGCGCTCGGTGTGCTGAGAGTGTCCTGCCTGGTCCTCTGTGCC
TGGTGGGGTGGGGGTGCCAGGTGTGTCCAGAGGAGCCCAGTTGGTAGTGAGGCAGCCATG
GGGCTAGAAGCACTGGTGCCCCTGGCCATGATAGTGGCCATCTTCCTGCTCCTGGTGGAC
CTGATGCACCGGCACCAACGCTGGGCTGCACGCTACCCGCCAGGTCCCCTGCCACTGCCC
GGGCTGGGCAACCTGCTGCATGTGGACTTCCAGAACACACCATACTGCTTCGACCAGTTG
CGGCGCCGCTTCGGGGACGTGTTCAACCTGCAGCTGGCCTGGACGCCGGTGGTCGTGCTC
AATGGGCTGGCGGCCGTGCGCGAGGCGATGGTGACCCGCGGCGAGGACACGGCCGACCGC
CCGCCTGCGCCCATCTACCAGGTCCTGGGCTTCGGGCCGCGTTCCCAAGGGGTGATCCTG
TCGCGCTATGGGCCCGCGTGGCGCGAGCAGAGGCGCTTCTCCGTGTCCACCTTGCGCAAC
TTGGGCCTGGGCAAGAAGTCGCTGGAGCAGTGGGTGACCGAGGAGGCCGCCTGCCTTTGT
GCCGCCTTCGCCGACCAAGCCGGACGCCCCTTTCGCCCCAACGGTCTCTTGGACAAAGCC
GTGAGCAACGTGATCGCCTCCCTCACCTGCGGGCGCCGCTTCGAGTACGACGACCCTCGC
TTCCTCAGGCTGCTGGACCTAGCTCAGGAGGGACTGAAGGAGGAGTCGGGCTTCCTGCGC
GAGGTGCTGAATGCTGTCCCCGTCCTCCCGCACATCCCAGCGCTGGCTGGCAAGGTCCTA
CGCTTCCAAAAGGCTTTCCTGACCCAGCTGGATGAGCTGCTAACTGAGCACAG

In [40]:
import pandas as pd

# Read the file and split it into lines
with open('fastaresults.txt', 'r') as file:
    lines = file.read().splitlines()

# Initialize empty lists for code and sequence
code_list = []
sequence_list = []

# Variables to track the current code and sequence
current_code = None
current_sequence = ''

# Iterate through the lines
for line in lines:
    if line.startswith('>ENA'):
        # Save the previous code and sequence, if any
        if current_code is not None:
            code_list.append(current_code)
            sequence_list.append(current_sequence)

        # Extract the code from the header
        current_code = line.split('|')[1]
        current_sequence = ''
    else:
        # Append the sequence line to the current sequence
        current_sequence += line

# Save the last code and sequence
if current_code is not None:
    code_list.append(current_code)
    sequence_list.append(current_sequence)

# Create a DataFrame from the lists
df = pd.DataFrame({'EMBL': code_list, 'nuc_sequence': sequence_list})


In [41]:
df

,EMBL,nuc_sequence
0,AY220845,CACGAGGGGAAGGGTCACGCGCTCGGTGTGCTGAGAGTGTCCTGCC...
1,AB593170,ATCCATTAAGTTTGGCCTTTGAGAGCAGTCGTCGCTCGCAAGCCCG...
2,CU639417,GATCTGTTGGTGGTTCCCTCGGCTTTGGACCTAGTCGCTCTGATTC...
3,KP715230,ATGAGGTGGCAAGAAATGGGCTACATTTTCTACCCCAGAAAACTAC...
4,LC013475,ATGACCCAGAGGGCTGGGGCTGCCATGCTGCCTTCAGCTCTGCTCC...
...,...,...
20400,AF152583,TGGGAATAATGAGTTCAAATGAACGAATATGTGGAACAAAGCATCA...
20401,AF086709,ATTTCCTCGAAATTAGATTCAGCTTATGCATTTTTTGGAAGGAATA...
20402,AB016901,GGAGGACACGTGGAAACGCAACCGCCAGGAGGAAAGGCGGGACACA...
20403,AL096814,GATCAGTCAGGCCAGTTTCCAGCCAGTGTTGGACAGTTAGTTGTTA...


In [45]:
mapping = pd.read_csv('uni_embl_map.csv').drop('Unnamed: 0',axis=1)

In [46]:
mapping

,Entry,EMBL_first_entry
0,A0A087X1C5,AY220845
1,A0A0B4J2F0,AB593170
2,A0A0B4J2F2,CU639417
3,A0A0C5B5G6,KP715230
4,A0A0K2S4Q6,LC013475
...,...,...
20417,Q9Y3F1,AF152583
20418,Q9Y6C7,AF086709
20419,Q9Y6Z2,AB016901
20420,X6R8D5,AL096814


In [75]:
mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20422 entries, 0 to 20421
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Entry             20422 non-null  object
 1   EMBL_first_entry  20408 non-null  object
dtypes: object(2)
memory usage: 319.2+ KB


In [52]:
merged = df.merge(mapping, left_on="EMBL", right_on="EMBL_first_entry", how='right').drop(['EMBL'],axis=1)[['Entry','EMBL_first_entry','nuc_sequence']].drop_duplicates()

In [63]:
merged[merged.duplicated(subset=['Entry','EMBL_first_entry'])]['Entry']

13796    Q9UHG0
23626    Q9UBP8
Name: Entry, dtype: object

In [70]:
merged[(merged['Entry']=='Q9UHG0')|(merged['Entry']=='Q9UBP8')]

,Entry,EMBL_first_entry,nuc_sequence
13795,Q9UHG0,AF181720,GAGCTCCCCTCCCCTCTCCAGCACGCGAGTGTCGGTAGGTACTACG...
13796,Q9UHG0,AF181720,GAGCTCCCCTCCCCTCTCCAGCACGCGAGTGTCGGTAGGTACTACG...
23625,Q9UBP8,AF181720,GAGCTCCCCTCCCCTCTCCAGCACGCGAGTGTCGGTAGGTACTACG...
23626,Q9UBP8,AF181720,GAGCTCCCCTCCCCTCTCCAGCACGCGAGTGTCGGTAGGTACTACG...


In [71]:
merged[(merged['Entry']=='Q9UHG0')|(merged['Entry']=='Q9UBP8')]['nuc_sequence'].str.len()

13795     4377
13796    34190
23625     4377
23626    34190
Name: nuc_sequence, dtype: int64

In [73]:
merged=merged.drop([13795,23626])

In [74]:
merged.to_csv('uniprot_nuc_sequences.csv')